In [1]:
from metrics.wallet import *

In [2]:
unconsistent_returns = {
    'ERR WRONGCERT', 'ERR UNSAT'
}

successfull_returns = {'SAT', 'UNSAT'}

is_consistent_by_xp=(lambda x: not x['Checked answer'] in unconsistent_returns)
is_consistent_by_input=(lambda df: len(set(df['Checked answer'].unique()) & successfull_returns) < 2)
is_success=(lambda x: x['Checked answer'] in successfull_returns)

analysis = Analysis(
    input_file='config/metrics_scalpel.yml',
    is_consistent_by_xp=is_consistent_by_xp,
    is_consistent_by_input=is_consistent_by_input,
    is_success=is_success
)

/Users/wattez/Documents/phd/metrics/metrics/wallet/dataframe/builder.py:141: UserWarning:

4 experiments are inconsistent and are declared as unsuccessful.



In [3]:
import re
family_re = re.compile(r'^XCSP\d\d/(.*?)/')
analysis.add_variable(
    new_var='family', 
    function=lambda x: family_re.match(x['input']).group(1),
    inplace=True
)

In [4]:
analysis.remove_variables(
    vars=['Category', 'Objective function'],
    inplace=True
)

In [5]:
analysis2 = analysis.add_variable(
    new_var='experiment_ware', 
    function=lambda x: x['experiment_ware'] + '-bis',
    inplace=False
)

In [6]:
analysis3 = analysis.add_analysis(
    analysis=analysis2,
    is_consistent_by_xp=is_consistent_by_xp,
    is_consistent_by_input=is_consistent_by_input,
    is_success=is_success
)

/Users/wattez/Documents/phd/metrics/metrics/wallet/dataframe/builder.py:141: UserWarning:

8 experiments are inconsistent and are declared as unsuccessful.



In [7]:
analysis_vbs = analysis.add_virtual_experiment_ware(
    function=find_best_cpu_time_input, 
    xp_ware_set=None, 
    name='vbew'
)

In [8]:
len(analysis_vbs.data_frame)

8100

In [9]:
analysis.experiment_wares

array(['BTD 19.07.01', 'cosoco 2.O parallel', 'cosoco 2.0 parallel',
       'choco-solver 2019-09-16', 'choco-solver 2019-09-20',
       'choco-solver 2019-06-14', 'choco-solver 2019-09-20 parallel',
       'choco-solver 2019-09-24 parallel',
       'choco-solver 2019-06-14 parallel',
       'choco-solver 2019-09-16 parallel', 'AbsCon 2019-07-23',
       'Concrete 3.12.3', 'Fun-sCOP hybrid+CryptoMiniSat (2019-06-15)',
       'Fun-sCOP order+GlueMiniSat (2019-06-15)', 'cosoco 2.0',
       'cosoco 2', 'Fun-sCOP hybrid+ManyGlucose (2019-09-22)',
       'Fun-sCOP order+ManyGlucose (2019-09-22)',
       'Fun-sCOP hybrid+ManyGlucose (2019-06-15)',
       'Fun-sCOP order+ManyGlucose (2019-06-15)',
       'choco-solver 2019-09-24', 'Concrete 3.10', 'PicatSAT 2019-09-12',
       'Concrete 3.12.2'], dtype=object)

In [10]:
analysis_no_para = analysis.filter_analysis(lambda x: 'parallel' not in x['experiment_ware'])\
                            .filter_analysis(lambda x: 'Many' not in x['experiment_ware'])
analysis_no_para.experiment_wares

/Users/wattez/Documents/phd/metrics/metrics/wallet/dataframe/builder.py:115: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



array(['BTD 19.07.01', 'choco-solver 2019-09-16',
       'choco-solver 2019-09-20', 'choco-solver 2019-06-14',
       'AbsCon 2019-07-23', 'Concrete 3.12.3',
       'Fun-sCOP hybrid+CryptoMiniSat (2019-06-15)',
       'Fun-sCOP order+GlueMiniSat (2019-06-15)', 'cosoco 2.0',
       'cosoco 2', 'choco-solver 2019-09-24', 'Concrete 3.10',
       'PicatSAT 2019-09-12', 'Concrete 3.12.2'], dtype=object)

In [11]:
analysis_clean = analysis_no_para.remove_experiment_wares({
    'Concrete 3.12.2', 
    'cosoco 2.0'
})

In [13]:
analysis_clean.experiment_wares

array(['BTD 19.07.01', 'choco-solver 2019-09-16',
       'choco-solver 2019-09-20', 'choco-solver 2019-06-14',
       'AbsCon 2019-07-23', 'Concrete 3.12.3',
       'Fun-sCOP hybrid+CryptoMiniSat (2019-06-15)',
       'Fun-sCOP order+GlueMiniSat (2019-06-15)', 'cosoco 2',
       'choco-solver 2019-09-24', 'Concrete 3.10', 'PicatSAT 2019-09-12'],
      dtype=object)

In [12]:
analysis.export('save.csv')